<a href="https://colab.research.google.com/github/Valent0296/allvent.github.io/blob/master/Test_4_tesi_ResNet_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 11.7 MB/s eta 0:00:00


In [ ]:
import torch
from torch import Tensor
import torch.nn as nn
from torch.nn import Transformer
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import DataLoader
import math
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from functools import partial
import os
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = models.resnet18(pretrained=True)
#print(model_ft)
#mod=model_ft.features
model_ft.add_module("conv1", nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3)))
model_ft.add_module("fc", nn.Linear(in_features=512, out_features=512, bias=True))
for idx, name in enumerate(model_ft.modules()):
  if name != "conv1":
    name.requires_grad_=False

  if name != "fc":
    name.requires_grad_=False

#print(model_ft)

NameError: name 'models' is not defined

In [ ]:
#######################
# Positional Encoding #
#######################
#1st version trial, future implementations will try diverse function
class PositionalEncoding( nn.Module ):

  def __init__( self, d_model: int, dropout: float=0.1, maxlen: int=8 ):
    super(PositionalEncoding, self).__init__()
    den = torch.exp( torch.arange( 0, d_model, 2 ) * ( -math.log(10000) / d_model ) )
    pos = torch.arange(0, maxlen).reshape( maxlen, 1 )
    self.pe = torch.zeros( (maxlen, d_model) )
    self.pe[:, 0::2] = torch.sin(pos*den)
    self.pe[:, 1::2] = torch.cos(pos*den)
    self.pe.unsqueeze_(-2) #BE CAREFUL TO THIS ONE SINCE IT CHANGES DIMENSION
    self.dropout = nn.Dropout(dropout)
    self.register_buffer("pos_enc", self.pe)

  def forward(self, ccn_embed: Tensor):
    return self.dropout( ccn_embed + self.pe[ :ccn_embed.size(1), 0,: ] ) #We encode the first tok_embed.size(0) positions


###################
# Input embedding #
###################
#1st approach: try passing a CCN output as an imput embedding, (ex: 400x180)
class SampleEmbedding( nn.Module ):

  def __init__(self, d_emb: int=512):
      super(SampleEmbedding, self).__init__()
      self.upscale = transforms.Resize((32,32))
      self.ccn = model_ft

  def forward(self, blocks: Tensor):
    blocks = self.upscale(blocks)
    return self.ccn(blocks)

###########
# Masking #
###########
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz))) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [ ]:
##############
# Tranformer #
##############
class TransformerModel(nn.Module):

  #ntokens referes to the latex vocabulary
  def __init__(self, ntokens: int, d_mod: int, d_hid: int ,n_head: int=8, dropout: float=0.5):
      super().__init__()
      self.model_type = "Transformer"
      self.nhead = n_head
      self.d_model = d_mod
      self.embedding = SampleEmbedding(d_mod)
      self.pos_encod = PositionalEncoding(d_mod, dropout)
      #self.tgt_encod = PositionalEncoding(10, dropout)
      self.tgt_embed = nn.Embedding(10,d_mod)
      encoder_layer = nn.TransformerEncoderLayer( d_mod, n_head, d_hid, batch_first=True )
      self.transformer_encoder = nn.TransformerEncoder(encoder_layer, n_head)
      decoder_layer = nn.TransformerDecoderLayer( d_mod, n_head, dropout=dropout, batch_first=True)
      self.transformer_decoder = nn.TransformerDecoder(decoder_layer, n_head )
      self.lin_soft_stack = nn.Sequential(
          nn.Linear(d_mod, ntokens),
          nn.Softmax(dim=1)
      )
      #self.init_weights()

  """def init_weights(self) -> None:
    initrange = 0.1
    self.embedding.weight.data.uniform_(-initrange, initrange)
    self.lin.bias.data.zero_()
    self.lin.weight.data.uniform_(-initrange, initrange)"""

  def forward(self, src: Tensor, tgt: Tensor, tgt_mask: Tensor) -> Tensor:
    """
    Args:
       src: Tensor, shape [seq_len, batch_size]

    Returns:
       output Tensor of shape [seq_len, batch_size, ntoken]
    """
    src_emb = torch.zeros((src.size(0),src.size(1),self.d_model))
    for i in range(src.size(1)):
      src_emb[:,i,:] = self.embedding(src[:,i,:,:].unsqueeze(1)) * math.sqrt(self.d_model)
    tgt = self.tgt_embed(tgt) * math.sqrt(self.d_model)
    src_emb = self.pos_encod(src_emb)
    tgt = self.pos_encod(tgt)
    memory = self.transformer_encoder(src_emb)
    deco = self.transformer_decoder(tgt, memory, tgt_mask=tgt_mask)
    output = self.lin_soft_stack(deco)
    return output

In [ ]:
model = TransformerModel(512, 512,1024)
"""for name in model.embedding.modules():
  print(name)"""

In [ ]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform = ToTensor(),
    #target_transform= Lambda( lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1) )
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    #target_transform= Lambda( lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1) )
)

train_val_data = torch.utils.data.random_split(data, [50000, 10000], generator=torch.Generator().manual_seed(42) )

In [ ]:
#Togliere la target transform sopra
labels_map = {
    0: "Zero",
    1: "One",
    2: "Two",
    3: "Three",
    4: "Four",
    5: "Five",
    6: "Six",
    7: "Seven",
    8: "Eight",
    9: "Nine",
}

figure = plt.figure(figsize=(8,8))
cols, rows = 3, 3
for i in range(1, cols*rows + 1):
  sample_idx = torch.randint(len(training_data), size=(1,)).item()
  img, label = training_data[sample_idx]
  figure.add_subplot(rows, cols, i)
  plt.title(labels_map[label])
  plt.axis("off")
  plt.imshow(img.squeeze(), cmap="gray")
plt.show()

In [ ]:

losses_train = np.zeros((10, 10))
losses_test = np.zeros(10)
batches_time = np.zeros(10)
accuracies = np.zeros((10,2))

#Define train and test loop
def train_loop(dataloader, model, loss_fn, optimizer):
  model.train()
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):

    ###FARE ATTENZIONE ALLA DIVISIBILITà DEI BATCH, PERCHè L'ULTIMO è SPESSO NON DIVISIBILE, TROVARE UN WORKAROUND
    tgt_mask = generate_square_subsequent_mask(8)
    #Compute prediction and loss
    if X.size(0) == 16:
      pred = model(X.view(2,8,28,28), y.view(2,8), tgt_mask)
    else:
      pred = model(X.view(8,8,28,28), y.view(8,8), tgt_mask)
    loss = loss_fn(torch.Tensor.view(pred, (X.size(0), -1)), y)

    #Backpropagation
    optimizer.zero_grad()
   # print("Sono qui ", batch)
    loss.backward()
    #print("E ora qui", batch)
    optimizer.step()
    correct += (torch.Tensor.view(pred, (X.size(0), -1)).argmax(1) == y).type(torch.float).sum().item()
    if batch % 100 == 0:
      b = (int)(batch/100)
      losses_train[t,b]=loss.item()
      if t == 1:
        batches_time[b] = batch*len(X)
      loss, current = loss.item(), batch * len(X)
      print(f"Funzione di loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

  accuracies[t][0]=100*correct/size

def validation_loop(valloader, model, loss_fn, optimizer, t,checkpoint_dir=None):
  # Validation loss
        val_loss = 0.0
        val_steps = 0
        size = len(valloader.dataset)
        correct = 0
        for X, y in valloader:
            with torch.no_grad():
                tgt_mask = generate_square_subsequent_mask(8)
                if X.size(0) == 16:
                  pred = model(X.view(2,8,28,28), y.view(2,8), tgt_mask)
                else:
                  pred = model(X.view(8,8,28,28), y.view(8,8), tgt_mask)
                val_loss += loss_fn(torch.Tensor.view(pred, (X.size(0), -1)), y).item()
                correct += (torch.Tensor.view(pred, (X.size(0), -1)).argmax(1) == y).type(torch.float).sum().item()
                val_steps += 1

        with tune.checkpoint_dir(t) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=correct / size)
        print("Finished Validation")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

     with torch.no_grad():
      for X, y in dataloader:
        tgt_mask = generate_square_subsequent_mask(8)
        if X.size(0) == 16:
          pred = model(X.view(2,8,28,28), y.view(2,8), tgt_mask)
        else:
          pred = model(X.view(8,8,28,28), y.view(8,8), tgt_mask)
        test_loss += loss_fn(torch.Tensor.view(pred, (X.size(0), -1)), y).item()
        correct += (torch.Tensor.view(pred, (X.size(0), -1)).argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Errore test: \n Accuratezza: {(100*correct):>0.1f}%, Media loss: {test_loss:>8f} \n")
    losses_test[t]=test_loss
    accuracies[t][1] = (100*correct)

In [ ]:
def train_transformer(config, checkpoint_dir=None):

  model = TransformerModel(512, 512,1024, n_head=config["heads"])

  if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=config["lr"], momentum = config["momentum"])
  #optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.98), eps=1e-9)
  train_dataloader = DataLoader(train_val_data[0], batch_size=batch_size, shuffle=True)
  val_dataloader = DataLoader(train_val_data[1], batch_size=batch_size, shuffle=True)
  test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

  for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, t)
    validation_loop(val_dataloader,  model, loss_fn, optimizer, t,checkpoint_dir=checkpoint_dir)
    test_loop(test_dataloader, model, loss_fn, t)
  print("Done!")

In [ ]:
#Definiamo la loss function e il metodo di minimizzazione
#learning_rate = 1e-3
#momentum = 0.9
batch_size = 64
epochs = 4

if torch.cuda.device_count() > 1:
  gpus_per_trial = 2
else:
  gpus_per_trial = 1

#Definisco il blocco per la scelta degli iperparametri
config = {
        "heads": tune.sample_from(lambda _: 2 ** np.random.randint(1, 4)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "momentum": tune.uniform(0.5, 0.9)
}

scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=epochs,
        grace_period=1,
        reduction_factor=2)

reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])

result = tune.run(
        partial(train_transformer),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=len(train_val_data[0]),
        scheduler=scheduler,
        progress_reporter=reporter)

best_trial = result.get_best_trial("loss", "min", "last")
print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(
best_trial.last_result["loss"]))
print("Best trial final validation accuracy: {}".format(
best_trial.last_result["accuracy"]))

best_trained_model = TransformerModel(512, 512,1024, n_head=config["heads"])
if gpus_per_trial > 1:
  best_trained_model = nn.DataParallel(best_trained_model)

best_checkpoint_dir = best_trial.checkpoint.value
model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
best_trained_model.load_state_dict(model_state)

In [ ]:
from matplotlib.pyplot import figure


print(len(losses_train))

fig1 = figure(figsize=(6,6), dpi=60)
ax1 = fig1.add_subplot(111)
ax1.set_xlabel("Batches")
ax1.set_ylabel("Loss value")
for i in range(epochs):
    ep = "Epoch {epo:5d}"
    ax1.plot(batches_time,losses_train[i,...], label=ep.format(epo=i+1))
ax1.legend()


fig2 = figure(figsize=(6,6), dpi=60)
ax2 = fig2.add_subplot(111)
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Avg test loss")
ax2.plot( range(epochs), losses_test)

fig3 = figure(figsize=(6,6), dpi=60)
ax3 = fig3.add_subplot(111)
ax3.set_xlabel("Epochs")
ax3.set_ylabel("Accuracy")
for i in range(2):
  acc = "Training accuracy" if i == 0 else "Test accuracy"
  ax3.plot(range(epochs),  accuracies[:,i], label=acc)
ax3.legend()